**Question - 2**

Used model is "Distill Bert". which is a model. The Model and Tokenizer from hugging face - "distilbert-base-uncased". First Creating the dataset by extracting data from youtube comments.

**Creating out custom dataset from youtube comments by using Google API**
Using 3 categories "Movies", "Technology" and "sports" as lables 0,1 and 2 respectivly.

In [2]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=6bbdddb2458face216d7a965320e4c6264d621db9c9688209d35798d4e51caa1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
import googleapiclient.discovery
import pandas as pd
from langdetect import detect
from sklearn.model_selection import train_test_split
import string
import seaborn as sns
import matplotlib.pyplot as plt
import os
import string
import re

In [22]:
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = "Using personal Google API key"

In [5]:
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

In [6]:
def clean_text(text):
    #Remove hyper links
    text = re.sub(r'https?:\/\/\S+', ' ', text)
    #Remove @mentions
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text)
    #Removing hashtags
    text = re.sub(r'#[A-Za-z0-9]+', ' ', text)
    #Remove anything that isn't a letter, number, or one of the punctuation marks listed
    text = re.sub(r"[^A-Za-z0-9#'?!,.]+", ' ', text)
    #The following regular expression are for removing apostrophes and expanding words
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    #Removing other items
    emoji_pattern = re.compile("["u"\U0001F600-\U0001F64F"  # emoticons
                                  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                  u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                "]+", flags=re.UNICODE)
    emoji_pattern.sub(r'', text)
    return text

In [7]:
def get_data(id,cat):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=id,
        maxResults=499)
    response = request.execute()
    comments = []
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        try:
            if len(comment['textDisplay'])>30:
                if detect(comment['textDisplay'])=='en':
                    text = clean_text(comment['textDisplay'])
                    comments.append([text, cat])
        except:
            continue
    df = pd.DataFrame(comments, columns=['text', 'label'])
    return df

In [8]:
cat=int(0)
# movie - 1
v_id="94TAFSMdkvk"
movie1_df = get_data(v_id,cat)

# movie - 2
v_id="3hqPFylPSns"
movie2_df = get_data(v_id,cat)

# movie - 3
v_id="9GEEMaVuFPE"
movie3_df = get_data(v_id,cat)

In [9]:
movie_df = pd.concat([movie1_df, movie2_df, movie3_df], ignore_index=True)
print(movie_df)

                                                  text  label
0    I remember in 2007 wondering whether Heath Led...      0
1    Bro I cried so hard when the most intense part...      0
2    Chills. Just utter chills, as the unplanned Ba...      0
3    I hate all Dark Night content. Who tf wants an...      0
4        Always makes me go cold, brilliant soundtrack      0
..                                                 ...    ...
208  I love this film, and I think every scene is b...      0
209  MOST OF THE SCENES WERE TO DARK TO SEE WHAT WA...      0
210              Groundhog Day with robots and aliens       0
211          Man this movie is definitely my favorite.      0
212  I would love a Prequal based around Rita and h...      0

[213 rows x 2 columns]


In [10]:
mtrain, mtest = train_test_split(movie_df, test_size=0.25, random_state=42)


In [11]:
# technology - 1
cat=int(1)
# iPhone 15 Review: Welcome to the Club!
v_id="0X0Jm8QValY"
technology1_df = get_data(v_id,cat)

# technology - 2
# Using Apple Vision Pro: What It’s Actually Like!
v_id="dtp6b76pMak"
technology2_df = get_data(v_id,cat)

# # technology - 3
# # The NEW Asus ROG Zephyrus G16 and G14
v_id="Qr0Xvw9cjCU"
technology3_df = get_data(v_id,cat)

In [12]:
technology_df = pd.concat([technology1_df, technology2_df, technology3_df],ignore_index=True)#, technology4_df, technology5_df], ignore_index=True)
print(technology_df)

                                                  text  label
0    you catch my attention. i wish i could grab so...      1
1    Due to the pink color I will be using the 15 p...      1
2     quot With the brightness of a box of crayons ...      1
3    I still find it funny talking about refresh ra...      1
4    My 13 is starting to crap out so idk if I wann...      1
..                                                 ...    ...
243  Boss, what do I think is the best performance ...      1
244  If the 16 inch coming with that silver color a...      1
245  g14 will work for a year and then it will brea...      1
246  How is a 4090 115 watts compared to a 4090 175...      1
247  I m on the 2020 G14 with the AMD Ryzen 9 4900H...      1

[248 rows x 2 columns]


In [13]:
ttrain, ttest = train_test_split(technology_df, test_size=0.25, random_state=42)


In [14]:
# Sports - 2
cat=int(2)

v_id="XxOh12Uhg08"
s1_df = get_data(v_id,cat)

# 2 - 2
v_id="s8WHY3Gusx0"
s2_df = get_data(v_id,cat)

# 2 - 3
v_id="qk7pZnxCX2o"
s3_df = get_data(v_id,cat)

In [15]:
s_df = pd.concat([s1_df, s2_df,s3_df], ignore_index=True)
print(s_df)

                                                  text  label
0                Horrible design and bad construction       2
1    the most overengineered thing Ive ever seen! i...      2
2     a href yes, accelerate to full speed in order...      2
3    the thing looks ugly as hell, but to be fair, ...      2
4        Everything I have seen . It is a hunk of junk      2
..                                                 ...    ...
188                                 a href where it is      2
189  Idk how they put all these trail cams up on my...      2
190  i have a HEARTATTACK when i saw this dog at a ...      2
191  How i think i look like when try to drift usin...      2
192  Whenever you see a shell car you know your abo...      2

[193 rows x 2 columns]


In [16]:
strain, stest = train_test_split(s_df, test_size=0.25, random_state=42)


Combining all dataframes

In [17]:
train_df = pd.concat([mtrain, ttrain, strain],ignore_index=True)
train_df.shape

(489, 2)

In [18]:
test_df = pd.concat([mtest, ttest, stest],ignore_index=True)
test_df.shape

(165, 2)

In [19]:
train_df.to_csv("train.csv", sep=',')

In [20]:
test_df.to_csv("test.csv", sep=',')


Now Finetuning the model and testing accuracy using the dataset we created

In [26]:
!pip install transformers datasets torch pandas

In [27]:
import pandas as pd
from datasets import Dataset

train_data = pd.read_csv('train.csv')
train_texts = train_data['text'].tolist()
train_labels = train_data['label'].tolist()
test_data = pd.read_csv('test.csv')
test_texts = test_data['text'].tolist()
test_labels = test_data['label'].tolist()

# Convert to our data to Hugging Face dataset format
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})


In [28]:
# Loading the tokeniser and tokeninsing the data
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/489 [00:00<?, ? examples/s]

Map:   0%|          | 0/165 [00:00<?, ? examples/s]

In [29]:
#Loading the model
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(train_labels)))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
!pip install accelerate -U

In [31]:
from datasets import load_metric
accuracy_metric = load_metric("accuracy")

<ipython-input-31-faaf2f42c7b8>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [32]:
#Defining compueting metrices
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [33]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    # per_device_train_batch_size=8,
    # logging_dir='./logs',
    # logging_steps=1,  # Log every 10 steps
    # evaluation_strategy='epoch',
    # learning_rate= 5e-5,
    # per_device_train_batch_size=4,  # Smaller batch size
    # logging_steps=1,  # Log after every step
)


In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

evaluation_results = trainer.evaluate()
print(evaluation_results)



Step,Training Loss


{'eval_loss': 0.6406182646751404, 'eval_accuracy': 0.8606060606060606, 'eval_runtime': 2.6534, 'eval_samples_per_second': 62.184, 'eval_steps_per_second': 7.914, 'epoch': 5.0}


Here is the accuracy of our model which is 86%. Distill-Bert model works with low number of parameters which makes it better for small and downstream instead of larger models like GPT, Mistral or Claude. For better accuracy than this we probabaly need more amount of data and pre-processing techniques for improving the quality of data. Although Youtube comments data does have a lot of outliers as well, so removing them will help in improving the model accuracy. Increasing amount of data is nessesary so that we can include more hyperparameters as well for fine tuning the model. And more.